In [1]:
import os
from utils import *
import optuna

In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_test, data_val = split_data2(data, 0, 0.2)

In [3]:
study_name = "P3Alpha"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2023-12-17 12:16:25,272] A new study created in RDB with name: P3Alpha


In [4]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

def objective(trial):
    topK = trial.suggest_int('topK', 1, 1000)
    alpha = trial.suggest_float('alpha', 0, 2)
    implicit = trial.suggest_categorical('implicit', [True, False])
    normalize_similarity = trial.suggest_categorical('normalize_similarity', [True, False])
    
    recommender = P3alphaRecommender(data_train, verbose=False)
    recommender.fit(
        topK=topK,
        alpha=alpha,
        implicit=implicit,
        normalize_similarity=normalize_similarity,
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=150)

[I 2023-12-17 12:17:05,527] Trial 0 finished with value: 0.04097575338794001 and parameters: {'topK': 621, 'alpha': 0.06135381025972064, 'implicit': False, 'normalize_similarity': False}. Best is trial 0 with value: 0.04097575338794001.
[I 2023-12-17 12:17:34,192] Trial 1 finished with value: 0.031209369961052617 and parameters: {'topK': 352, 'alpha': 1.150452460643985, 'implicit': False, 'normalize_similarity': False}. Best is trial 0 with value: 0.04097575338794001.
[I 2023-12-17 12:18:08,602] Trial 2 finished with value: 0.023600899116106046 and parameters: {'topK': 666, 'alpha': 1.3067741985874441, 'implicit': False, 'normalize_similarity': False}. Best is trial 0 with value: 0.04097575338794001.
[I 2023-12-17 12:18:35,983] Trial 3 finished with value: 0.024317757984782157 and parameters: {'topK': 384, 'alpha': 1.230323099886746, 'implicit': False, 'normalize_similarity': False}. Best is trial 0 with value: 0.04097575338794001.
[I 2023-12-17 12:19:06,941] Trial 4 finished with valu

KeyboardInterrupt: 

In [ ]:
study.best_params

In [ ]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

recommender = P3alphaRecommender(data_train, verbose=False)
recommender.fit(**study.best_params)

In [ ]:
pr, re, ev_map, ev_ndcg, ev_auc = evaluator(recommender, data_train, data_val)
print("Precision: {:.8f}".format(pr))
print("Recall: {:.8f}".format(re))
print("MAP: {:.8f}".format(ev_map))

In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)